In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *

/tmp/ipykernel_53001/1286269870.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
sg  = spark.read.parquet("/dados10t/datalake/standard/vigivac/SG/sg_long_09072022_v9.parquet/")
vac  = spark.read.parquet("/dados10t/datalake/standard/vigivac/vacinacao/vacinacao07072022_pessoasunicas_5doses_v17.parquet/")
srag  = spark.read.parquet("/dados10t/datalake/standard/vigivac/srag/dbsrag_tb_influd_long_09072022_v8.parquet/")

In [3]:
id_vigivac = spark.read.parquet("/dados10t/datalake/standard/vigivac/base_analise/id_paciente_to_id_vigivac/id_vigivac_id_paciente_09072022.parquet/")
id_vigivac_v2 = id_vigivac.select("id_paciente","id_vigvac","vacinacao")

In [4]:
vac_raw  = spark.read.parquet("/dados10t/datalake/raw/vigivac/vacinacao/vacinacao07072022_v5.parquet/")
sg_raw  = spark.read.parquet("/dados10t/datalake/raw/vigivac/sg/dbnotifica_tb_sindrome_gripal_from_pandas_09072022.parquet/")
srag_raw  = spark.read.parquet("/dados10t/datalake/raw/vigivac/srag/dbsrag_tb_influd_from_pandas_09072022.parquet/")

In [5]:
vac_raw_dt = vac_raw.select("id_paciente","paciente_datanascimento")
sg_raw_dt = sg_raw.select("id_paciente","datanascimento")
srag_raw_dt = srag_raw.select("id_paciente","dt_nasc")

In [6]:
srag_raw_dt_grouped = srag_raw_dt.groupBy("id_paciente").agg( F.first("dt_nasc",ignorenulls=True).alias("dt_nasc") )
sg_raw_dt_grouped = sg_raw_dt.groupBy("id_paciente").agg( F.first("datanascimento",ignorenulls=True).alias("datanascimento") )
vac_raw_dt_grouped = vac_raw_dt.groupBy("id_paciente").agg( F.first("paciente_datanascimento",ignorenulls=True).alias("paciente_datanascimento") )

In [7]:
sg.printSchema()

root
 |-- dt_datanotificacao: date (nullable = true)
 |-- dt_datainiciosintomas: date (nullable = true)
 |-- dt_datateste: date (nullable = true)
 |-- condicoes: string (nullable = true)
 |-- sintomas: string (nullable = true)
 |-- classificacaofinal: string (nullable = true)
 |-- dt_dataencerramento: date (nullable = true)
 |-- evolucaocaso: string (nullable = true)
 |-- tipoteste: string (nullable = true)
 |-- resultadoteste: string (nullable = true)
 |-- codigotipoteste: string (nullable = true)
 |-- codigoresultadoteste: string (nullable = true)
 |-- estadonotificacao: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- idade: double (nullable = true)
 |-- codigo_ibge_resid: string (nullable = true)
 |-- profissionalsaude: string (nullable = true)
 |-- cbo: string (nullable = true)
 |-- evolucaocaso_2: string (nullable = true)
 |-- classificacaofinal_2: string (nullable = true)
 |-- sexo_2: string (nullable = true)
 |-- racac

In [8]:
sg = sg.withColumn("diffdays",F.datediff("dt_dataencerramento","dt_datanotificacao") )

In [9]:
sg.select("dt_datanotificacao","dt_dataencerramento","diffdays").show(10)

+------------------+-------------------+--------+
|dt_datanotificacao|dt_dataencerramento|diffdays|
+------------------+-------------------+--------+
|        2021-06-02|               null|    null|
|        2021-03-26|         2021-05-24|      59|
|        2022-01-13|               null|    null|
|        2022-01-24|               null|    null|
|        2021-01-26|         2021-02-02|       7|
|        2022-01-20|               null|    null|
|        2021-04-12|               null|    null|
|        2021-02-19|               null|    null|
|        2022-03-11|               null|    null|
|        2021-05-09|               null|    null|
+------------------+-------------------+--------+
only showing top 10 rows



In [10]:
#sg.filter(F.col("dt_dataencerramento").isNotNull()).count()

In [11]:
#sg.count()

In [12]:
srag.select("id_mn_resi","co_mun_res").show(10)

+--------------------+----------+
|          id_mn_resi|co_mun_res|
+--------------------+----------+
|      BELO HORIZONTE|    310620|
|           SAO PAULO|    355030|
|             ARACUAI|    310340|
|CONSELHEIRO LAFAIETE|    311830|
|             ERECHIM|    430700|
|           FORTALEZA|    230440|
|          VILA VELHA|    320520|
|      BELO HORIZONTE|    310620|
|              MARABA|    150420|
|             CUBATAO|    351350|
+--------------------+----------+
only showing top 10 rows



In [13]:
srag.select("co_un_inte","co_uni_not").show(20)

+----------+----------+
|co_un_inte|co_uni_not|
+----------+----------+
|      null|   0023051|
|   6668143|   6668143|
|   2134276|   2134276|
|   2098326|   8000956|
|   2249472|   2249472|
|      null|   2527057|
|   7257406|   7257406|
|   7866801|   7866801|
|   2615797|   2615797|
|   2078473|   2078473|
|   2263823|   2263823|
|   2576341|   2576341|
|   7107366|   7107366|
|   0012521|   0012521|
|   9383298|   9383298|
|   2089777|   2089777|
|   2352516|   2352516|
|   2022877|   2022877|
|   2200902|   2200902|
|   2079127|   2079127|
+----------+----------+
only showing top 20 rows



In [14]:
vac.printSchema()

root
 |-- d_sexo: string (nullable = true)
 |-- d_codibge: string (nullable = true)
 |-- d_idade: double (nullable = true)
 |-- d_racacor: string (nullable = true)
 |-- d_paciente_endereco_uf: string (nullable = true)
 |-- d_total_regitros_dedup: double (nullable = true)
 |-- d1_nome_vacina: string (nullable = true)
 |-- d1_data_aplicacao: date (nullable = true)
 |-- d1_estab_mun_codigo: string (nullable = true)
 |-- d1_estab_valor: string (nullable = true)
 |-- d1_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d1_vacina_categoria_nome: string (nullable = true)
 |-- d1_vacina_dose: string (nullable = true)
 |-- d2_nome_vacina: string (nullable = true)
 |-- d2_data_aplicacao: date (nullable = true)
 |-- d2_estab_mun_codigo: string (nullable = true)
 |-- d2_estab_valor: string (nullable = true)
 |-- d2_vacina_grupoAtendimento_nome: string (nullable = true)
 |-- d2_vacina_categoria_nome: string (nullable = true)
 |-- d2_vacina_dose: string (nullable = true)
 |-- d3_nome_vacin

### seleção de variáveis de pessoa de cada banco

#### vacina

In [15]:
vac_person = vac.select("d_sexo","d_codibge","id_vigvac","inconsistente","d_idade","d_racacor")

#### srag

In [16]:
srag_person = srag.select("cs_sexo","co_mun_res","id_vigvac","nu_idade_n","tp_idade","cs_raca","cs_etinia")#,"co_mun_res","co_un_inte","co_uni_not")

#### sg

In [17]:
sg_person = sg.select("sexo_2","municipio","id_vigvac","idade","racacor_2")#,"municipio")

In [18]:
id_vigivac_v2.count()

188537327

In [19]:
vac_person.count()

183759941

In [20]:
srag_person.count()

2487085

In [21]:
sg_person.count()

82896519

In [22]:
id_vigivac_v2.select("id_vigvac").distinct().count()

188537327

In [23]:
vac_person.select("id_vigvac").distinct().count()

183759941

In [24]:
srag_person.select("id_vigvac").distinct().count()

2320559

In [25]:
sg_person.select("id_vigvac").distinct().count()

53109190

In [26]:
for nome in sg_person.columns:
    if nome == "id_vigvac":
        pass
    else:
        sg_person = sg_person.withColumnRenamed(nome,"sg_"+nome)

In [27]:
for nome in vac_person.columns:
    if nome == "id_vigvac":
        pass
    else:
        vac_person = vac_person.withColumnRenamed(nome,"vac_"+nome)

In [28]:
for nome in srag_person.columns:
    if nome == "id_vigvac":
        pass
    else:
        srag_person = srag_person.withColumnRenamed(nome,"srag_"+nome)

In [29]:
sg_person.printSchema()

root
 |-- sg_sexo_2: string (nullable = true)
 |-- sg_municipio: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- sg_idade: double (nullable = true)
 |-- sg_racacor_2: string (nullable = true)



In [30]:
sg_person.groupBy("sg_sexo_2").count().orderBy(F.desc("count")).show(300,False) 

+----------+--------+
|sg_sexo_2 |count   |
+----------+--------+
|Feminino  |45511625|
|Masculino |37301161|
|Indefinido|82953   |
|null      |780     |
+----------+--------+



In [31]:
sg_person.groupBy("sg_racacor_2").count().orderBy(F.desc("count")).show(300,False) 

+-------------+--------+
|sg_racacor_2 |count   |
+-------------+--------+
|Branca       |31615421|
|Parda        |25570173|
|null         |19866924|
|Preta        |3218746 |
|Amarela      |2456329 |
|Indigena     |168924  |
|Não informado|2       |
+-------------+--------+



In [32]:
srag_person.printSchema()

root
 |-- srag_cs_sexo: string (nullable = true)
 |-- srag_co_mun_res: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- srag_nu_idade_n: decimal(10,0) (nullable = true)
 |-- srag_tp_idade: string (nullable = true)
 |-- srag_cs_raca: short (nullable = true)
 |-- srag_cs_etinia: string (nullable = true)



In [33]:
srag_person.groupBy("srag_cs_sexo").count().orderBy(F.desc("count")).show(300,False) 

+------------+-------+
|srag_cs_sexo|count  |
+------------+-------+
|M           |1344547|
|F           |1142513|
|I           |25     |
+------------+-------+



In [34]:
srag_person.groupBy("srag_cs_etinia").count().orderBy(F.desc("count")).show(10,False) 

+----------------------------------+-------+
|srag_cs_etinia                    |count  |
+----------------------------------+-------+
|null                              |2484368|
|KAINGANG (CAINGANGUE)             |412    |
|TERENA                            |307    |
|GUARANI KAIOWA (PAI TAVYTERA)     |250    |
|KOCAMA (COCAMA, KOKAMA)           |105    |
|GUARANI                           |92     |
|MAKUXI (MACUXI, MACHUSI, PEMON)   |79     |
|XAVANTE (A'UWE, AKWE, AWEN, AKWEN)|68     |
|MURA                              |63     |
|TICUNA (TIKUNA, TUKUNA, MAGUTA)   |62     |
+----------------------------------+-------+
only showing top 10 rows



In [35]:
 srag_person.groupBy("srag_cs_raca").count().orderBy(F.desc("count")).show(10,False) 

+------------+-------+
|srag_cs_raca|count  |
+------------+-------+
|1           |1098563|
|4           |821187 |
|9           |403505 |
|2           |107781 |
|null        |27250  |
|3           |24113  |
|5           |4686   |
+------------+-------+



In [36]:
vac_person.printSchema()

root
 |-- vac_d_sexo: string (nullable = true)
 |-- vac_d_codibge: string (nullable = true)
 |-- id_vigvac: long (nullable = true)
 |-- vac_inconsistente: string (nullable = true)
 |-- vac_d_idade: double (nullable = true)
 |-- vac_d_racacor: string (nullable = true)



In [37]:
vac_person.groupBy("vac_d_sexo").count().orderBy(F.desc("count")).show(300,False) 

+----------+--------+
|vac_d_sexo|count   |
+----------+--------+
|F         |95510810|
|M         |88245694|
|I         |3437    |
+----------+--------+



In [38]:
vac_person.groupBy("vac_d_racacor").count().orderBy(F.desc("count")).show(300,False) 

+-------------+--------+
|vac_d_racacor|count   |
+-------------+--------+
|BRANCA       |61517395|
|null         |45074568|
|PARDA        |42199385|
|AMARELA      |26604921|
|PRETA        |7806738 |
|INDIGENA     |556934  |
+-------------+--------+



### otmizar depois para pegar o valor com maior frequencia entre quem tem o mesmo id

In [39]:
srag_person_grouped = srag_person.groupBy("id_vigvac").agg( F.first("srag_cs_sexo",ignorenulls=True).alias("srag_cs_sexo"), 
                                                            F.first("srag_cs_etinia",ignorenulls=True).alias("srag_cs_etinia"),
                                                          F.first("srag_cs_raca",ignorenulls=True).alias("srag_cs_raca"))

In [40]:
sg_person_grouped = sg_person.groupBy("id_vigvac").agg( F.first("sg_sexo_2",ignorenulls=True).alias("sg_sexo_2"), F.first("sg_racacor_2",ignorenulls=True).alias("sg_racacor_2") )

In [41]:
sg_person_grouped.groupBy("sg_sexo_2").count().orderBy(F.desc("count")).show(300,False) 

+----------+--------+
|sg_sexo_2 |count   |
+----------+--------+
|Feminino  |28535800|
|Masculino |24522435|
|Indefinido|50320   |
|null      |635     |
+----------+--------+



In [42]:
sg_person_grouped.groupBy("sg_racacor_2").count().orderBy(F.desc("count")).show(300,False) 

+-------------+--------+
|sg_racacor_2 |count   |
+-------------+--------+
|Branca       |20966789|
|Parda        |18222142|
|null         |9771404 |
|Preta        |2261902 |
|Amarela      |1766380 |
|Indigena     |120571  |
|Não informado|2       |
+-------------+--------+



In [43]:
tb_pessoa = id_vigivac_v2.join(vac_person,['id_vigvac'] , 'left' )
tb_pessoa = tb_pessoa.join(srag_person_grouped,['id_vigvac'] , 'left' )
tb_pessoa = tb_pessoa.join(sg_person_grouped,['id_vigvac'] , 'left' ).cache()

In [44]:
tb_pessoa.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- id_paciente: string (nullable = true)
 |-- vacinacao: string (nullable = true)
 |-- vac_d_sexo: string (nullable = true)
 |-- vac_d_codibge: string (nullable = true)
 |-- vac_inconsistente: string (nullable = true)
 |-- vac_d_idade: double (nullable = true)
 |-- vac_d_racacor: string (nullable = true)
 |-- srag_cs_sexo: string (nullable = true)
 |-- srag_cs_etinia: string (nullable = true)
 |-- srag_cs_raca: short (nullable = true)
 |-- sg_sexo_2: string (nullable = true)
 |-- sg_racacor_2: string (nullable = true)



In [47]:
tb_pessoa.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- id_paciente: string (nullable = true)
 |-- vacinacao: string (nullable = true)
 |-- vac_d_sexo: string (nullable = true)
 |-- vac_d_codibge: string (nullable = true)
 |-- vac_inconsistente: string (nullable = true)
 |-- vac_d_idade: double (nullable = true)
 |-- vac_d_racacor: string (nullable = true)
 |-- srag_cs_sexo: string (nullable = true)
 |-- srag_cs_etinia: string (nullable = true)
 |-- sg_sexo_2: string (nullable = true)
 |-- sg_racacor_2: string (nullable = true)



In [45]:
tb_pessoa.count() #188537327

188537327

In [46]:
tb_pessoa.select("id_vigvac").distinct().count() #188537327

188537327

In [47]:
tb_pessoa.show(10,False)

+---------+----------------------------------------------------------------+---------+----------+-------------+-----------------+-----------+-------------+------------+--------------+------------+---------+------------+
|id_vigvac|id_paciente                                                     |vacinacao|vac_d_sexo|vac_d_codibge|vac_inconsistente|vac_d_idade|vac_d_racacor|srag_cs_sexo|srag_cs_etinia|srag_cs_raca|sg_sexo_2|sg_racacor_2|
+---------+----------------------------------------------------------------+---------+----------+-------------+-----------------+-----------+-------------+------------+--------------+------------+---------+------------+
|26       |0001ff0fe81e6fe93017f0916e2736582d073e67395aedcf3d3442528f2f1279|vacinacao|F         |291300       |null             |8.0        |PARDA        |null        |null          |null        |null     |null        |
|29       |00021308008b1c05812fd61abbfb8037eb4f823878df0da6d76880995376e983|vacinacao|F         |412010       |null     

In [48]:
for nome in sg_raw_dt_grouped.columns:
    if nome == "id_paciente":
        pass
    else:
        sg_raw_dt_grouped = sg_raw_dt_grouped.withColumnRenamed(nome,"sg_raw_"+nome)
        
for nome in srag_raw_dt_grouped.columns:
    if nome == "id_paciente":
        pass
    else:
        srag_raw_dt_grouped = srag_raw_dt_grouped.withColumnRenamed(nome,"srag_raw_"+nome)
        

for nome in vac_raw_dt_grouped.columns:
    if nome == "id_paciente":
        pass
    else:
        vac_raw_dt_grouped = vac_raw_dt_grouped.withColumnRenamed(nome,"vac_raw_"+nome)

In [49]:
vac_raw_dt_grouped.printSchema()

root
 |-- id_paciente: string (nullable = true)
 |-- vac_raw_paciente_datanascimento: string (nullable = true)



In [50]:
tb_pessoa = tb_pessoa.join(sg_raw_dt_grouped,['id_paciente'] , 'left' )
tb_pessoa = tb_pessoa.join(srag_raw_dt_grouped,['id_paciente'] , 'left' )
tb_pessoa_v2 = tb_pessoa.join(vac_raw_dt_grouped,['id_paciente'] , 'left' )#.cache()

In [51]:
tb_pessoa_v2.show(15,False)

+----------------------------------------------------------------+---------+---------+----------+-------------+-----------------+-----------+-------------+------------+--------------+------------+---------+------------+------------------------+----------------+-------------------------------+
|id_paciente                                                     |id_vigvac|vacinacao|vac_d_sexo|vac_d_codibge|vac_inconsistente|vac_d_idade|vac_d_racacor|srag_cs_sexo|srag_cs_etinia|srag_cs_raca|sg_sexo_2|sg_racacor_2|sg_raw_datanascimento   |srag_raw_dt_nasc|vac_raw_paciente_datanascimento|
+----------------------------------------------------------------+---------+---------+----------+-------------+-----------------+-----------+-------------+------------+--------------+------------+---------+------------+------------------------+----------------+-------------------------------+
|00000c08e215dfabf66a513ada8cce634347971b4693334ed94f4bb78bacb587|0        |vacinacao|M         |211130       |null   

In [52]:
tb_pessoa_v2.count() #188537327

188537327

In [53]:
tb_pessoa_v3 = tb_pessoa_v2.drop("id_paciente")

In [54]:
tb_pessoa_v3.printSchema()

root
 |-- id_vigvac: long (nullable = true)
 |-- vacinacao: string (nullable = true)
 |-- vac_d_sexo: string (nullable = true)
 |-- vac_d_codibge: string (nullable = true)
 |-- vac_inconsistente: string (nullable = true)
 |-- vac_d_idade: double (nullable = true)
 |-- vac_d_racacor: string (nullable = true)
 |-- srag_cs_sexo: string (nullable = true)
 |-- srag_cs_etinia: string (nullable = true)
 |-- srag_cs_raca: short (nullable = true)
 |-- sg_sexo_2: string (nullable = true)
 |-- sg_racacor_2: string (nullable = true)
 |-- sg_raw_datanascimento: string (nullable = true)
 |-- srag_raw_dt_nasc: string (nullable = true)
 |-- vac_raw_paciente_datanascimento: string (nullable = true)



In [55]:
tb_pessoa_v3.write.parquet("/dados10t/datalake/raw/omop/tabelas_input/tabela_pessoa_07072022_vigivac_nao_padronizada_v2.parquet")

In [12]:
vac_person.show()

+------+---------+------------+-------------+-------+---------+
|d_sexo|d_codibge|   id_vigvac|inconsistente|d_idade|d_racacor|
+------+---------+------------+-------------+-------+---------+
|     M|   290210|463856467968|         null|   33.0|    PARDA|
|     M|   355030|463856467969|         null|   13.0|     null|
|     F|   292260|463856467970|         null|   30.0|   BRANCA|
|     M|   150180|463856467971|         null|   13.0|    PARDA|
|     M|   330455|463856467972|         null|   34.0|     null|
|     F|   530010|463856467973|         null|   14.0|    PARDA|
|     F|   320140|463856467974|         null|   35.0|   BRANCA|
|     F|   410690|463856467975|         null|   38.0|  AMARELA|
|     M|   500790|463856467976|         null|   46.0|    PARDA|
|     F|   291072|463856467977|         null|   57.0|    PARDA|
|     M|   150680|463856467978|         null|   50.0|    PARDA|
|     M|   432270|463856467980|         null|   45.0|     null|
|     F|   250400|463856467981|         

In [13]:
srag_person.show()

+-------+----------+-------------+----------+--------+-------+---------+
|cs_sexo|co_mun_res|    id_vigvac|nu_idade_n|tp_idade|cs_raca|cs_etinia|
+-------+----------+-------------+----------+--------+-------+---------+
|      F|    310620|1511828488212|        54|       3|      1|     null|
|      F|    355030|1511828488233|        86|       3|      9|     null|
|      M|    310340|1511828488237|        54|       3|      9|     null|
|      F|    311830|1511828488269|        74|       3|      1|     null|
|      F|    430700|1511828488443|        96|       3|      1|     null|
|      F|    230440|1511828488504|        87|       3|      4|     null|
|      F|    320520|1511828488524|        62|       3|      4|     null|
|      M|    310620|1511828488600|        69|       3|      4|     null|
|      M|    150420|1511828488669|        13|       3|      4|     null|
|      F|    351350|1511828488848|        41|       3|      1|     null|
|      M|    431390|1511828488850|        54|      

In [14]:
sg_person.show()

+---------+-------------------+------------+-----+---------+
|   sexo_2|          municipio|   id_vigvac|idade|racacor_2|
+---------+-------------------+------------+-----+---------+
|Masculino|     rio de janeiro|463856467979| 43.0|   Branca|
|Masculino|          são paulo|463856467983| 70.0|     null|
| Feminino|           jandaíra|463856467985| 45.0|    Parda|
| Feminino|       itapetininga|463856467990| 23.0|   Branca|
| Feminino|      caxias do sul|463856467992| 26.0|    Parda|
| Feminino|     paraíba do sul|463856467998| 48.0|   Branca|
|Masculino|      novo hamburgo|463856467999| 23.0|   Branca|
|Masculino|      novo hamburgo|463856467999| 23.0|   Branca|
|Masculino|      novo hamburgo|463856467999| 23.0|     null|
|Masculino|      novo hamburgo|463856467999| 23.0|     null|
|Masculino|      novo hamburgo|463856467999| 23.0|   Branca|
|Masculino|      novo hamburgo|463856467999| 23.0|   Branca|
| Feminino|     rio de janeiro|463856468001| 63.0|     null|
| Feminino|     rio de j